In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier


sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [8]:
station_population=ss.read.csv('station_population.csv',header=True)

In [9]:
station_population.show(5)

+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
| id|                name|      lat|      long|dock_count|    city|installation_date| _c7| _c8|
+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
|  2|San Jose Diridon ...|37.329732|-121.90178|        27|San Jose|           8/6/13|null|null|
|  3|San Jose Civic Ce...|37.330698|-121.88898|        15|San Jose|           8/5/13|null|null|
|  4|Santa Clara at Al...|37.333988| -121.8949|        11|San Jose|           8/6/13|null|null|
|  5|    Adobe on Almaden|37.331415| -121.8932|        19|San Jose|           8/5/13|null|null|
|  6|    San Pedro Square|37.336721|-121.89407|        15|San Jose|           8/7/13|null|null|
+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
only showing top 5 rows



In [13]:
status=ss.read.csv('status.csv',header=True)
status.show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|              2|             25|2013/08/29 12:06:01|
|         2|              2|             25|2013/08/29 12:07:01|
|         2|              2|             25|2013/08/29 12:08:01|
|         2|              2|             25|2013/08/29 12:09:01|
|         2|              2|             25|2013/08/29 12:10:01|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



In [21]:
trip=ss.read.csv('trip.csv',header=True)

In [34]:
trip=ss.read.csv('trip.csv',header=True)

In [98]:
trip=sc.textFile('trip.csv')

In [100]:
tripheader = trip.first() 
header = sc.parallelize([tripheader])
tripdata= trip.subtract(header)

In [108]:
trip_rdd=tripdata.map(lambda x: x.split(','))\
.map(lambda x:[int(x[0]),int(x[1]),int(x[2].split(' ')[-1].replace(':','')),\
              x[2].split(' ')[0],x[3],int(x[4]),int(x[5].split(' ')[-1].replace(':','')),\
               x[6],int(x[7]),int(x[8]),x[9],int(x[10])])

In [112]:
from pyspark.sql.types import *
tripschema = StructType([
    StructField('id',IntegerType(),True),
    StructField('duration',IntegerType(),True),
    StructField('start_date_time',IntegerType(),True),
    StructField("start_date",StringType(),True),
    StructField('start_station_name',StringType(),True),
    StructField('start_station_id',IntegerType(),True),
    StructField('end_date_time',IntegerType(),True),
    StructField('end_station_name',StringType(),True),
    StructField( 'end_station_id',IntegerType(),True),
    StructField( 'bike_id',IntegerType(),True),
    StructField('subscription_type',StringType(),True),
    StructField( 'zip_code',IntegerType(),True),
])



In [114]:
trip_df=ss.createDataFrame(trip_rdd, tripschema)

In [121]:
trip_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date_time: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date_time: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: integer (nullable = true)



In [122]:
trip_df.show(5)

+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|start_date_time|start_date|  start_station_name|start_station_id|end_date_time|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+
|4329|     142|           1211| 8/29/2013|      Market at 10th|              67|         1214|      Market at 10th|            67|    319|       Subscriber|   94103|
|4510|     166|           1331| 8/29/2013|San Jose Civic Ce...|               3|         1334| San Salvador at 1st|             8|    661|       Subscriber|   95112|
|4714|     240|           1621| 8/29/2013|         MLK Library|              11|         1625|SJSU 4th at San C...|            12|     88|       Subscriber|   95112|
|465

In [174]:
def peak_hours(x):
    if (x>700) and (x<=1000):
        return 'peak_hours'
    elif (x>=1600) and (x<=2000):
        return 'peak_hours'
    else:
        return 'normal_hours'

In [175]:
from pyspark.sql.functions import *

In [176]:
trip_df=trip_df.dropna()

In [177]:
check_peak=udf(peak_hours)

In [167]:
trip_df.show(5)

+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|start_date_time|start_date|  start_station_name|start_station_id|end_date_time|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+
|4329|     142|           1211| 8/29/2013|      Market at 10th|              67|         1214|      Market at 10th|            67|    319|       Subscriber|   94103|
|4510|     166|           1331| 8/29/2013|San Jose Civic Ce...|               3|         1334| San Salvador at 1st|             8|    661|       Subscriber|   95112|
|4714|     240|           1621| 8/29/2013|         MLK Library|              11|         1625|SJSU 4th at San C...|            12|     88|       Subscriber|   95112|
|465

In [166]:
trip_df.select('id',check_peak('start_date_time')).show(5)

Py4JJavaError: An error occurred while calling o1433.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 99.0 failed 1 times, most recent failure: Lost task 0.0 in stage 99.0 (TID 105, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pys

In [172]:
complete=trip_df.join(station_population,trip_df.start_station_id==station_population.id,'left_outer')

In [173]:
complete.show(3)

+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+---+--------------------+---------+----------+----------+-------------+-----------------+------+----+
|  id|duration|start_date_time|start_date|  start_station_name|start_station_id|end_date_time|    end_station_name|end_station_id|bike_id|subscription_type|zip_code| id|                name|      lat|      long|dock_count|         city|installation_date|   _c7| _c8|
+----+--------+---------------+----------+--------------------+----------------+-------------+--------------------+--------------+-------+-----------------+--------+---+--------------------+---------+----------+----------+-------------+-----------------+------+----+
|4329|     142|           1211| 8/29/2013|      Market at 10th|              67|         1214|      Market at 10th|            67|    319|       Subscriber|   94103| 67|      Market at 10th|37.776619